In [4]:
library(phyloseq)
library(ggplot2)
library(vegan)
library(readr)
library(data.table)
library(dplyr)
library(stats)
library(plotly)
library(RColorBrewer)
library(ape)
library(nlme)
library(micropower)
library(reshape2)

In [3]:
#CREATE PHYLOSEQ OBJECT - No NA's
setwd("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/batrun_finaldata_for_phyloseq_Oct2017")

biom <- import_biom("BIOMfiles/final.alldata.biom")
map <- import_qiime_sample_data("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/mapfile.complete.nona.txt")
tree <- read_tree("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/batrun_finaldata_for_phyloseq_Oct2017/make_phylo.tre")
batrun <- merge_phyloseq(biom,tree,map)

#FILTER DATA
batrun_nt <- prune_samples((sample_sums(batrun) > 1000), batrun)

#TRANSFORM DATA
batrun_t <- prune_samples((sampleSums(batrun_nt) > 1000), batrun_nt) %>%
    transform_sample_counts(function(x) x / sum(x))

Warning message in strsplit(msg, "\n"):
“input string 1 is invalid in this locale”Warning message:
“'sampleSums' is deprecated.
Use 'sample_sums' instead.
See help("Deprecated") and help("phyloseq-deprecated").”

In [4]:
saveRDS(batrun_nt, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/batrun_nt.nona.rds")
saveRDS(batrun_t, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/batrun_t.nona.rds")

In [5]:
fecal = subset_samples(batrun_t, SampleType=="FecalFTA")
oral = subset_samples(batrun_t, SampleType=="TongueEtOH")
skin = subset_samples(batrun_t, SampleType=="SkinEtOH")

In [17]:
fecal = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/fecal_t.nona.rds")
oral = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/oral_t.nona.rds")
skin = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/skin_t.nona.rds")

In [7]:
fecal.df = data.frame(sample_data(fecal))
oral.df = data.frame(sample_data(oral))
skin.df = data.frame(sample_data(skin))

write.csv(fecal.df, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/fecal_t.nona.df.csv")
write.csv(oral.df, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/oral_t.nona.df.csv")
write.csv(skin.df, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/skin_t.nona.df.csv")

In [8]:
fecal.wunif = distance(fecal, method = "wunifrac", type = "samples")
oral.wunif = distance(oral, method = "wunifrac", type = "samples")
skin.wunif = distance(skin, method = "wunifrac", type = "samples")

Warning message in UniFrac(physeq, weighted = TRUE, ...):
“Randomly assigning root as -- CACGTAGGGGGCAAGCGTTGTCCGGATTCATTGGGCGTAAAGAGCTCGTAGGCGGTTCGGTAAGTCGGGTGTGAAACCTCCAGGCTCAACCTGGAGACGCCACCCGATACTGCTGTGACTAGAGTCCAGTAGGGGAGCGTGGAATTCCTG -- in the phylogenetic tree in the data you provided.”Warning message in UniFrac(physeq, weighted = TRUE, ...):
“Randomly assigning root as -- TACGTAGGGTGCGAGCGTTATCCGGAATTATTGGGCGTAAAGAGCTTGTAGGCGGTTTGTCGCGTCTGCTGTGAAAGCCCGGGGCTTAACTCCGGGTCTGCAGTGGGTACGGGCAGGCTAGAGTGATGTAGGGGTAACTGGAATTCCTGG -- in the phylogenetic tree in the data you provided.”Warning message in UniFrac(physeq, weighted = TRUE, ...):
“Randomly assigning root as -- GACAGAGGGTGCAAACGTTGTTCGGAATTACTGGGCGTAAAGCGCGTGTAGGCGGCCCGAAAAGTTGGATGTGAAAGCCCTGGGCTTAACCTAGGAAGTGCATTCAAAACTGTCGGGCTTGAGTTCTGGAGAGGAAGGCGGAATTCTCGG -- in the phylogenetic tree in the data you provided.”

In [9]:
saveRDS(fecal.wunif, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/fecal.nona.wunif.rds")
saveRDS(oral.wunif, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/oral.nona.wunif.rds")
saveRDS(skin.wunif, "/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/skin.nona.wunif.rds")


In [6]:
fecal.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/fecal_t.nona.df.csv")
oral.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/oral_t.nona.df.csv")
skin.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/RDSfiles/skin_t.nona.df.csv")

In [1]:
fecal.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/fecal.nona.wunif.rds")
oral.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/oral.nona.wunif.rds")
skin.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/skin.nona.wunif.rds")


### PERMANOVA of taxonomic ranks, nested

In [15]:
# FECAL
adonis2(fecal.wunif ~ Host_Family, nperm=999, data=fecal.df)
adonis2(fecal.wunif ~ Host_Family:Host_Genus, nperm=999, data=fecal.df)
adonis2(fecal.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=fecal.df)


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,8,2.6256,0.09992504,4.66278,0.001
Residual,336,23.6501,0.90007496,NA,NA
Total,344,26.2757,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,17,4.973166,0.1892687,4.490561,0.001
Residual,327,21.302530,0.8107313,NA,NA
Total,344,26.275696,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,25,6.388973,0.2431514,4.099383,0.001
Residual,319,19.886723,0.7568486,NA,NA
Total,344,26.275696,1.0000000,NA,NA


In [16]:
# ORAL
adonis2(oral.wunif ~ Host_Family, nperm=999, data=oral.df)
adonis2(oral.wunif ~ Host_Family:Host_Genus, nperm=999, data=oral.df)
adonis2(oral.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=oral.df)


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,8,4.407509,0.3004263,16.42616,0.001
Residual,306,10.263338,0.6995737,NA,NA
Total,314,14.670847,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,17,5.122967,0.3491937,9.373939,0.001
Residual,297,9.547880,0.6508063,NA,NA
Total,314,14.670847,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,27,7.181519,0.4895095,10.19276,0.001
Residual,287,7.489328,0.5104905,NA,NA
Total,314,14.670847,1.0000000,NA,NA


In [17]:
# SKIN
adonis2(skin.wunif ~ Host_Family, nperm=999, data=skin.df)
adonis2(skin.wunif ~ Host_Family:Host_Genus, nperm=999, data=skin.df)
adonis2(skin.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=skin.df)


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,8,4.114379,0.1993104,11.10821,0.001
Residual,357,16.528692,0.8006896,NA,NA
Total,365,20.643071,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,16,5.435218,0.2632951,7.79569,0.001
Residual,349,15.207852,0.7367049,NA,NA
Total,365,20.643071,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,27,8.269915,0.4006146,8.367072,0.001
Residual,338,12.373156,0.5993854,NA,NA
Total,365,20.643071,1.0000000,NA,NA


### PERMANOVA of host variables

In [7]:
# FECAL
adonis2(fecal.wunif ~ Site_name, nperm=999, data=fecal.df)
adonis2(fecal.wunif ~ Diet, strata = Site_name, nperm=999, data=fecal.df)
adonis2(fecal.wunif ~ Site_name + Host_species + Diet, by="margin", nperm=999, data=fecal.df)
adonis2(fecal.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=fecal.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,16,4.17091,0.1587364,3.868106,0.001
Residual,328,22.10479,0.8412636,NA,NA
Total,344,26.27570,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,1.120455,0.04264225,15.27777,0.001
Residual,343,25.155242,0.95735775,NA,NA
Total,344,26.275696,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,15,1.901436e+00,7.236483e-02,2.142628,0.001
Host_species,23,3.679929e+00,1.400507e-01,2.704381,0.001
Diet,0,3.552714e-15,1.352091e-16,Inf,NA
Residual,304,1.798529e+01,6.844837e-01,NA,NA
Total,344,2.627570e+01,1.000000e+00,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.26657107,0.010145157,1.8175397,0.039
Sex,1,0.04227201,0.001608788,0.5764396,0.854
Weight,1,0.95914109,0.036502975,13.0792663,0.001
Residual,340,24.93320046,0.948907313,NA,NA
Total,344,26.27569639,1.000000000,NA,NA


In [34]:
#FDR p-value correction
p = c("0.039","0.859","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.0585 0.8590 0.0030

In [14]:
# ORAL
adonis2(oral.wunif ~ Site_name, nperm=999, data=oral.df)
adonis2(oral.wunif ~ Diet, strata = Site_name, nperm=999, data=oral.df)
adonis2(oral.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=oral.df)
adonis2(oral.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=oral.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,15,3.67107,0.2502289,6.65256,0.001
Residual,299,10.99978,0.7497711,NA,NA
Total,314,14.67085,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,2.08935,0.1424151,51.97843,0.001
Residual,313,12.58150,0.8575849,NA,NA
Total,314,14.67085,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,13,0.7024475,0.0478805,2.181478,0.001
Host_species,25,4.2128964,0.2871611,6.803324,0.001
Residual,274,6.7868803,0.4626100,NA,NA
Total,314,14.6708472,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.2535351,0.017281558,3.083883,0.003
Sex,1,0.1389525,0.009471337,3.380308,0.009
Weight,1,1.3041164,0.088891691,31.725328,0.001
Residual,310,12.7430073,0.868593829,NA,NA
Total,314,14.6708472,1.000000000,NA,NA


In [36]:
#FDR p-value correction
p = c("0.004","0.010","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.006 0.010 0.003

In [15]:
# SKIN
adonis2(skin.wunif ~ Site_name, nperm=999, data=skin.df)
adonis2(skin.wunif ~ Diet, strata = Site_name, nperm=999, data=skin.df)
adonis2(skin.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=skin.df)
adonis2(skin.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=skin.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,16,8.893148,0.4308055,16.5092,0.001
Residual,349,11.749922,0.5691945,NA,NA
Total,365,20.643071,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,1.135076,0.05498582,21.17941,0.001
Residual,364,19.507995,0.94501418,NA,NA
Total,365,20.643071,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,15,3.762177,0.1822489,9.408016,0.001
Host_species,26,3.138944,0.1520580,4.528561,0.001
Residual,323,8.610978,0.4171365,NA,NA
Total,365,20.643071,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.43168635,0.020911925,4.065434,0.001
Sex,1,0.07307444,0.003539902,1.376367,0.161
Weight,1,0.75401896,0.036526492,14.202045,0.001
Residual,361,19.16631261,0.928462277,NA,NA
Total,365,20.64307090,1.000000000,NA,NA


In [8]:
#FDR p-value correction
p = c("0.001","0.161","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.0015 0.1610 0.0015

### Restricting samples to species for which N >= 10

In [2]:
fecal.10.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/fecal.10.nona.wunif.rds")
oral.10.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/oral.10.nona.wunif.rds")
skin.10.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/skin.10.nona.wunif.rds")

In [3]:
batfecal.10.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/batfecal.10.df.csv")
batoral.10.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/batoral.10.df.csv")
batskin.10.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/batskin.10.df.csv")

#### FECAL

In [48]:
adonis2(fecal.10.wunif ~ Host_Family, nperm=999, data=batfecal.10.df)
adonis2(fecal.10.wunif ~ Host_Family:Host_Genus, nperm=999, data=batfecal.10.df)
adonis2(fecal.10.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=batfecal.10.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,7,3.816932,0.1095103,5.498859,0.001
Residual,313,31.037606,0.8904897,NA,NA
Total,320,34.854539,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,9,5.492922,0.1575956,6.464596,0.001
Residual,311,29.361616,0.8424044,NA,NA
Total,320,34.854539,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,16,7.398947,0.2122807,5.120268,0.001
Residual,304,27.455592,0.7877193,NA,NA
Total,320,34.854539,1.0000000,NA,NA


In [6]:
adonis2(fecal.10.wunif ~ Site_name, nperm=999, data=batfecal.10.df)
adonis2(fecal.10.wunif ~ Diet, strata=Site_name, nperm=999, data=batfecal.10.df)
adonis2(fecal.10.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=batfecal.10.df)
adonis2(fecal.10.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=batfecal.10.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,14,5.772704,0.1656227,4.338613,0.001
Residual,306,29.081834,0.8343773,NA,NA
Total,320,34.854539,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,1.912089,0.05485911,18.51582,0.001
Residual,319,32.942449,0.94514089,NA,NA
Total,320,34.854539,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,13,2.411586,0.06919002,2.155503,0.001
Host_species,15,4.037829,0.11584800,3.127849,0.001
Residual,291,25.044006,0.71852926,NA,NA
Total,320,34.854539,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.39556180,0.011348932,1.914271,0.023
Sex,1,0.08654554,0.002483049,0.837652,0.571
Weight,1,1.61269586,0.046269322,15.608869,0.001
Residual,316,32.64886679,0.936717803,NA,NA
Total,320,34.85453855,1.000000000,NA,NA


In [20]:
#FDR p-value correction
p = c("0.023","0.571","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.0345 0.5710 0.0030

#### ORAL

In [49]:
adonis2(oral.10.wunif ~ Host_Family, nperm=999, data=batoral.10.df)
adonis2(oral.10.wunif ~ Host_Family:Host_Genus, nperm=999, data=batoral.10.df)
adonis2(oral.10.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=batoral.10.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,7,5.746699,0.3160845,18.28868,0.001
Residual,277,12.434197,0.6839155,NA,NA
Total,284,18.180896,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,9,6.049637,0.3327469,15.2375,0.001
Residual,275,12.131258,0.6672531,NA,NA
Total,284,18.180896,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,16,8.560470,0.4708497,14.90452,0.001
Residual,268,9.620426,0.5291503,NA,NA
Total,284,18.180896,1.0000000,NA,NA


In [7]:
adonis2(oral.10.wunif ~ Site_name, nperm=999, data=batoral.10.df)
adonis2(oral.10.wunif ~ Diet, strata=Site_name, nperm=999, data=batoral.10.df)
adonis2(oral.10.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=batoral.10.df)
adonis2(oral.10.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=batoral.10.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,13,5.24214,0.2883323,8.445823,0.001
Residual,271,12.93876,0.7116677,NA,NA
Total,284,18.18090,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,3.070469,0.1688844,57.50617,0.001
Residual,283,15.110427,0.8311156,NA,NA
Total,284,18.180896,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,12,0.8957853,0.04927069,2.190358,0.001
Host_species,15,4.2141157,0.23178812,8.243424,0.001
Residual,256,8.7246404,0.47987957,NA,NA
Total,284,18.1808958,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.3797058,0.02088488,3.431244,0.002
Sex,1,0.1917880,0.01054888,3.466218,0.011
Weight,1,1.8124485,0.09968973,32.756695,0.001
Residual,280,15.4925760,0.85213491,NA,NA
Total,284,18.1808958,1.00000000,NA,NA


In [21]:
#FDR p-value correction
p = c("0.002","0.011","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.003 0.011 0.003

#### SKIN

In [53]:
adonis2(skin.10.wunif ~ Host_Family, nperm=999, data=batskin.10.df)
adonis2(skin.10.wunif ~ Host_Family:Host_Genus, nperm=999, data=batskin.10.df)
adonis2(skin.10.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=batskin.10.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,7,3.631193,0.2061637,11.98356,0.001
Residual,323,13.981963,0.7938363,NA,NA
Total,330,17.613157,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,9,4.357978,0.2474274,11.72633,0.001
Residual,321,13.255179,0.7525726,NA,NA
Total,330,17.613157,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,16,6.618895,0.3757927,11.81487,0.001
Residual,314,10.994262,0.6242073,NA,NA
Total,330,17.613157,1.0000000,NA,NA


In [8]:
adonis2(skin.10.wunif ~ Site_name, nperm=999, data=batskin.10.df)
adonis2(skin.10.wunif ~ Diet, srtata=Site_name, nperm=999, data=batskin.10.df)
adonis2(skin.10.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=batskin.10.df)
adonis2(skin.10.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=batskin.10.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,15,7.966549,0.4523067,17.34263,0.001
Residual,315,9.646608,0.5476933,NA,NA
Total,330,17.613157,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,0.9984843,0.05668969,19.77176,0.001
Residual,329,16.6146724,0.94331031,NA,NA
Total,330,17.6131567,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,14,3.442658,0.1954594,9.768949,0.001
Host_species,15,2.095004,0.1189454,5.548500,0.001
Residual,300,7.551604,0.4287479,NA,NA
Total,330,17.613157,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.40235253,0.022843863,4.019966,0.001
Sex,1,0.06535965,0.003710842,1.306037,0.173
Weight,1,0.62649701,0.035569831,12.518857,0.001
Residual,326,16.31443040,0.926263855,NA,NA
Total,330,17.61315668,1.000000000,NA,NA


In [22]:
#FDR p-value correction
p = c("0.001","0.173","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.0015 0.1730 0.0015

### Restricting samples to species for which N >= 20

In [10]:
batfecal.20.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/fecal.20.nona.wunif.rds")
batoral.20.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/oral.20.nona.wunif.rds")
batskin.20.wunif = readRDS("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/Bdiv/nona_updates_19April2019/skin.20.nona.wunif.rds")


In [11]:
batfecal.20.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/batfecal.20.df.csv")
batoral.20.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/batoral.20.df.csv")
batskin.20.df = read.csv("/gpfs/data/gilbert-lab/Lutz/Batmicrobiome/Nature_data_results/CSVfiles/batskin.20.df.csv")

#### FECAL

In [63]:
adonis2(batfecal.20.wunif ~ Host_Family, nperm=999, data=batfecal.20.df)
adonis2(batfecal.20.wunif ~ Host_Family:Host_Genus, nperm=999, data=batfecal.20.df)
adonis2(batfecal.20.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=batfecal.20.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,4,0.9665238,0.1109064,8.139344,0.001
Residual,261,7.7482504,0.8890936,NA,NA
Total,265,8.7147742,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,5,1.321112,0.1515946,9.291451,0.001
Residual,260,7.393662,0.8484054,NA,NA
Total,265,8.714774,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,10,1.580232,0.181328,5.648004,0.001
Residual,255,7.134542,0.818672,NA,NA
Total,265,8.714774,1.000000,NA,NA


In [27]:
adonis2(batfecal.20.wunif ~ Site_name, nperm=999, data=batfecal.20.df)
adonis2(batfecal.20.wunif ~ Diet, strata=Site_name, nperm=999, data=batfecal.20.df)
adonis2(batfecal.20.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=batfecal.20.df)
adonis2(batfecal.20.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=batfecal.20.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,14,1.797673,0.2062788,4.659425,0.001
Residual,251,6.917101,0.7937212,NA,NA
Total,265,8.714774,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,0.682547,0.07832068,22.43368,0.001
Residual,264,8.032227,0.92167932,NA,NA
Total,265,8.714774,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,13,0.8065112,0.09254528,2.372542,0.001
Host_species,9,0.5890702,0.06759443,2.503060,0.001
Residual,242,6.3280308,0.72612676,NA,NA
Total,265,8.7147742,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.11521846,0.013221050,1.8797779,0.046
Sex,1,0.02024005,0.002322498,0.6604287,0.752
Weight,1,0.53664807,0.061579114,17.5107214,0.001
Residual,261,7.99882221,0.917846180,NA,NA
Total,265,8.71477420,1.000000000,NA,NA


In [28]:
#FDR p-value correction
p = c("0.046","0.752","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.069 0.752 0.003

#### ORAL

In [67]:
adonis2(batoral.20.wunif ~ Host_Family, nperm=999, data=batoral.20.df)
adonis2(batoral.20.wunif ~ Host_Family:Host_Genus, nperm=999, data=batoral.20.df)
adonis2(batoral.20.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=batoral.20.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,4,2.528800,0.3142558,26.12137,0.001
Residual,228,5.518148,0.6857442,NA,NA
Total,232,8.046948,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,5,2.573272,0.3197824,21.34334,0.001
Residual,227,5.473676,0.6802176,NA,NA
Total,232,8.046948,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,10,3.419519,0.4249461,16.40507,0.001
Residual,222,4.627429,0.5750539,NA,NA
Total,232,8.046948,1.0000000,NA,NA


In [13]:
adonis2(batoral.20.wunif ~ Site_name, nperm=999, data=batoral.20.df)
adonis2(batoral.20.wunif ~ Diet, strata=Site_name, nperm=999, data=batoral.20.df)
adonis2(batoral.20.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=batoral.20.df)
adonis2(batoral.20.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=batoral.20.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,13,2.675917,0.3325381,8.392971,0.001
Residual,219,5.371031,0.6674619,NA,NA
Total,232,8.046948,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,1.761307,0.2188789,64.7288,0.001
Residual,231,6.285641,0.7811211,NA,NA
Total,232,8.046948,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,12,0.496012,0.06163976,2.101025,0.001
Host_species,9,1.239614,0.15404772,7.001067,0.001
Residual,210,4.131417,0.51341417,NA,NA
Total,232,8.046948,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.1752010,0.02177235,3.082505,0.005
Sex,1,0.1099954,0.01366920,3.870541,0.008
Weight,1,1.1133880,0.13836152,39.178137,0.001
Residual,228,6.4794419,0.80520488,NA,NA
Total,232,8.0469482,1.00000000,NA,NA


In [29]:
#FDR p-value correction
p = c("0.005","0.008","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.0075 0.0080 0.0030

#### SKIN

In [70]:
adonis2(batskin.20.wunif ~ Host_Family, nperm=999, data=batskin.20.df)
adonis2(batskin.20.wunif ~ Host_Family:Host_Genus, nperm=999, data=batskin.20.df)
adonis2(batskin.20.wunif ~ Host_Family:Host_Genus:Host_species, nperm=999, data=batskin.20.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family,4,1.414328,0.1486588,11.61204,0.001
Residual,266,8.099593,0.8513412,NA,NA
Total,270,9.513921,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus,5,1.683453,0.1769462,11.39434,0.001
Residual,265,7.830469,0.8230538,NA,NA
Total,270,9.513921,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Host_Family:Host_Genus:Host_species,10,2.430933,0.2555133,8.92339,0.001
Residual,260,7.082988,0.7444867,NA,NA
Total,270,9.513921,1.0000000,NA,NA


In [32]:
adonis2(batskin.20.wunif ~ Site_name, nperm=999, data=batskin.20.df)
adonis2(batskin.20.wunif ~ Diet, strata=Site_name, nperm=999, data=batskin.20.df)
adonis2(batskin.20.wunif ~ Site_name + Host_species, by="margin", nperm=999, data=batskin.20.df)
adonis2(batskin.20.wunif ~ Age + Sex + Weight, strata = Host_species, by="margin", nperm=999, data=batskin.20.df)

,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,15,4.138439,0.4349877,13.08784,0.001
Residual,255,5.375482,0.5650123,NA,NA
Total,270,9.513921,1.0000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Diet,1,0.5959921,0.06264421,17.97748,0.001
Residual,269,8.9179291,0.93735579,NA,NA
Total,270,9.5139212,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Site_name,14,2.5306252,0.26599182,9.767829,0.001
Host_species,9,0.8231197,0.08651739,4.942182,0.001
Residual,246,4.5523628,0.47849490,NA,NA
Total,270,9.5139212,1.00000000,NA,NA


,Df,SumOfSqs,R2,F,Pr(>F)
Age,2,0.22960947,0.024134052,3.498618,0.001
Sex,1,0.05823363,0.006120887,1.774642,0.058
Weight,1,0.40199659,0.042253513,12.250650,0.001
Residual,266,8.72860566,0.917456164,NA,NA
Total,270,9.51392122,1.000000000,NA,NA


In [33]:
#FDR p-value correction
p = c("0.001","0.058","0.001")
p.adjust(p, method = "fdr", n = length(p))

[1] 0.0015 0.0580 0.0015